Benson = EGUB/03658

51.6201, -1.0983

In [ ]:
import datetime as dt
import glob
import os

import cfgrib
import requests
import xarray as xr
from dask.distributed import wait, Client as DaskClient
# from ecmwf.opendata import Client as ECMWFClient

In [ ]:
dask_client = DaskClient(n_workers=4, threads_per_worker=2)

## Dynamical - GFS

In [ ]:
ds_gfs = xr.open_zarr("https://data.dynamical.org/noaa/gfs/forecast/latest.zarr?email=email@email.com")

ds_gfs

In [ ]:
variable_list = [
    "pressure_reduced_to_mean_sea_level",
    "relative_humidity_2m", 
    "temperature_2m", 
    "total_cloud_cover_atmosphere", 
    "wind_u_10m",
    "wind_v_10m"
    ]

temp_var_list = [
    "pressure_reduced_to_mean_sea_level", 
    "relative_humidity_2m", 
    "total_cloud_cover_atmosphere"
]

ds_gfs_subset = ds_gfs[temp_var_list].sel(
    latitude=slice(52, 51.25), longitude=slice(-1.5, -0.75), init_time=slice("2022-10-24", "2025-10-24")
    )
ds_gfs_subset

In [ ]:
with DaskClient():
    ds_gfs_subset.to_zarr("data/gfs_benson.zarr", mode="a")

In [ ]:
# pressure_reduced_to_mean_sea_level, relative_humidity_2m, total_cloud_cover_atmosphere

xr.open_zarr("data/gfs_benson.zarr")#.total_cloud_cover_atmosphere.isel(init_time=-1, latitude=-1, longitude=-1).plot()

## ECMWF - IFS

Data starts 2023-01-18

- url to 20240131 00z = [yyyymmdd]/[run]z/0p4-beta/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20240228 00z = [yyyymmdd]/[run]z/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20250123 18z = [yyyymmdd]/[run]z/[ifs|aifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2
- url to 20250209 06z = [yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url to 20250225 00z = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/[experimental]/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2     * Experimental for AIFS *
- url to 20250227 18z =[yyyymmdd]/[run]z/[ifs]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2    * missing AIFS *
- url to now = [yyyymmdd]/[run]z/[ifs|aifs-single]/0p25/oper/[yyyymmdd][run]0000-[step]h-oper-fc.grib2

In [ ]:
def request_url_process_data(url, grib_file_path, nc_file_path, date_str, run, step):
    response = requests.get(url)

    if response.status_code == 200:
        with open(grib_file_path, "wb") as f:
            f.write(response.content)
        print(f"Downloaded: {grib_file_path}")

    else:
        print(f"Couldn't download {date_str} {run}z {step}h - Status code: {response.status_code}")
        return False

    ds_list = cfgrib.open_datasets(grib_file_path)
    ds_list_final = []
    vars = ['t2m', 'd2m', 'msl', 'u10', 'v10', 'tcc']
    for ds_n in ds_list:
        var_names = [var for var in vars if var in ds_n.data_vars]
        if len(var_names) > 0:
            if 'heightAboveGround' in ds_n.coords:
                ds_n = ds_n.drop_vars('heightAboveGround')
            ds_list_final.append(ds_n[var_names])

    ds = xr.merge(ds_list_final, compat="no_conflicts")
    ds = ds.sel(latitude=slice(52, 51.25), longitude=slice(-1.5, -0.75))
    ds.to_netcdf(nc_file_path)

    ds.close()
    for f in glob.glob(f"{grib_file_path}*"):
        os.remove(f)


In [ ]:
url_base = "https://ecmwf-forecasts.s3.eu-central-1.amazonaws.com"

start_dt = dt.datetime(2023, 1, 19)
end_dt = dt.datetime(2023, 1, 20)

delta = dt.timedelta(days=1)

dates = []
while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    dates.append(start_dt)
    # increment start date by timedelta
    start_dt += delta

date_str_list = [date.strftime("%Y%m%d") for date in dates]
runs = ["00", "12"]
steps = list(range(0, 121, 3))

futures = []
ds_list = []
for date_str in date_str_list:
    for run in runs:
        date_folder = f"data/ecmwf_ifs/{date_str}_{run}z"
        os.makedirs(date_folder, exist_ok=True)

        for step in steps:
            url = f"{url_base}/{date_str}/{run}z/0p4-beta/oper/{date_str}{run}0000-{step}h-oper-fc.grib2"
            grib_file_path = os.path.join(date_folder, f"ifs_{date_str}_{run}z_{step}h.grib2")
            nc_file_path = os.path.join(date_folder, f"ifs_{date_str}_{run}z_{step}h.nc")

            future = dask_client.submit(request_url_process_data, url, grib_file_path, nc_file_path, date_str, run, step)
            futures.append(future)

wait(futures)